In [1]:
def gen_trigrams():
  """
      Generates all trigrams for characters from `trigram_chars`
  """
  trigram_chars="0123456789abcdefghijklmnopqrstuvwxyz"
  t3=[''.join(x) for x in itertools.product(trigram_chars,repeat=3)] #len(words)>=3
  t2_start=['#'+''.join(x) for x in itertools.product(trigram_chars,repeat=2)] #len(words)==2
  t2_end=[''.join(x)+'#' for x in itertools.product(trigram_chars,repeat=2)] #len(words)==2
  t1=['#'+''.join(x)+'#' for x in itertools.product(trigram_chars)] #len(words)==1
  trigrams=t3+t2_start+t2_end+t1
  vocab_size=len(trigrams)
  trigram_map=dict(zip(trigrams,range(1,vocab_size+1))) # trigram to index mapping, indices starting from 1
  return trigram_map

In [2]:
def sentences_to_bag_of_trigrams(sentences):
  """
      Converts a sentence to bag-of-trigrams
      `sentences`: list of strings
      `trigram_BOW`: return value, (len(sentences),len(trigram_map)) size array
  """
  trigram_map=gen_trigrams()
  trigram_BOW=np.zeros((len(sentences),len(trigram_map))) # one row for each sentence
  filter_pat=r'[\!"#&\(\)\*\+,-\./:;<=>\?\[\\\]\^_`\{\|\}~\t\n]' # characters to filter out from the input
  for j,sent in enumerate(sentences):
      sent=re.sub(fiter_pat, '', sent).lower() # filter out special characters from input
      sent=re.sub(r"(\s)\s+", r"\1", sent) # reduce multiple whitespaces to single whitespace
      words=sent.split(' ')
      indices=collections.defaultdict(int)
      for word in words:
          word='#'+word+'#'
          #print(word)
          for k in range(len(word)-2): # generate all trigrams for word `word` and update `indices`
              trig=word[k:k+3]
              idx=trigram_map.get(trig, 0)
              #print(trig,idx)
              indices[idx]=indices[idx]+1     
      for key,val in indices.items(): #covert `indices` dict to np array
          trigram_BOW[j,key]=val
  return trigram_BOW

In [3]:
def FC_layer(X,INP_NEURONS,OUT_NEURONS, X_is_sparse=False):
     """
         Create a Fully Connected layer
         `X`: input array/activations of previous layer
         `INP_NEURONS`: number of neurons in previous layer
         `OUT_NEURONS`: number of neurons in this layer
         `X_is_sparse`: bool value to indicate if input `X` is sparse or not.
                        Default value is False

     """
     limit=np.sqrt(6.0/(INP_NEURONS+OUT_NEURONS))
     W=tf.Variable(tf.random_uniform(
         (INP_NEURONS,OUT_NEURONS), -limit, limit), #weight init
         name="weight")
     b=tf.Variable(tf.random_uniform((OUT_NEURONS),-limit, limit), name="bias") # bias
     prod=tf.sparse_tensor_dense_matmul(X,W) if X_is_sparse else tf.matmul(X,W) #linear transformation
     return tf.nn.tanh(prod+b) # non-linear (tanh) transformation

In [4]:
def cosine_similarity(self,A,B):
     """
         Function to calculate cosine similarity between two (batches of) vectors
         `A` and  `B`: Inputs (shape => [batch_size,vector_size])
         `sim`: Return value, cosine similarity of A and B
     """
     Anorm=tf.nn.l2_normalize(A, dim=1) # normalize A
     Bnorm=tf.nn.l2_normalize(B, dim=1) # normalize B
     sim=tf.reduce_sum(Anorm*Bnorm, axis=1) # dot product of normalized A and normalized B
     return sim

In [5]:
smax=tf.nn.softmax_cross_entropy_with_logits(logits=batch_cosine_similarities, labels=Y)

NameError: name 'tf' is not defined